定义球坐标下的全局符号变量$r,\theta,\phi$

In [4]:
import math
import numpy as np
import scipy
import sympy as sp
r,theta,phi,r1,theta1,phi1 = sp.symbols('r theta phi r1 theta1 phi1')



使用STO-3G基组，定义C原子和H原子的轨道系数和指数

In [5]:
alphas_1s_H = np.array([0.3425250914E+01,0.6239137298E+00,0.1688554040E+00])
coefficients_1s_H = np.array([0.1543289673E+00,0.5353281423E+00,0.4446345422E+00])

alphas_1s_C = np.array([0.7161683735E+02,0.1304509632E+02,0.3530512160E+01])
coefficients_1s_C = np.array([0.1543289673E+00,0.5353281423E+00,0.4446345422E+00])
alphas_2s_C = np.array([0.2941249355E+01,0.6834830964E+00,0.2222899159E+00])
coefficients_2s_C = np.array([-0.9996722919E-01,0.3995128261E+00,0.7001154689E+00])
alphas_2p_C = np.array([0.2941249355E+01,0.6834830964E+00,0.2222899159E+00])
coefficients_2p_C = np.array([0.1559162750E+00,0.6076837186E+00,0.3919573931E+00])

定义$s$轨道的形式
$$\phi_{1s}=\left(\frac{2\alpha}{\pi}\right)^{\frac{3}{4}}e^{-\alpha r^2} $$

In [6]:
def s_gaussian_function(coefficient,alpha):
    
    s_function = 0
    for i in range(0,3):
        s_function = s_function + (coefficient[i] * sp.exp(-alpha[i] * r ** 2) * ((2*alpha[i]/sp.pi) **(3/4)))
    return s_function


In [7]:
print(s_gaussian_function(coefficients_1s_H, alphas_1s_H))


0.653490442100107*exp(-3.425250914*r**2)/pi**0.75 + 0.632027505218341*exp(-0.6239137298*r**2)/pi**0.75 + 0.196975367085776*exp(-0.168855404*r**2)/pi**0.75


定义$2p_x,2p_y,2p_z$轨道的形式
$$\phi_{2p_x}=\left(\frac{128\alpha^5}{\pi^3}\right)^{\frac{1}{4}} r\sin\theta\cos\phi e^{-\alpha r^2} $$
$$\phi_{2p_y}=\left(\frac{128\alpha^5}{\pi^3}\right)^{\frac{1}{4}} r\sin\theta\sin\phi e^{-\alpha r^2} $$
$$\phi_{2p_z}=\left(\frac{128\alpha^5}{\pi^3}\right)^{\frac{1}{4}} r\cos\theta e^{-\alpha r^2} $$

In [8]:
def px_gaussian_function(coefficient,alpha):
    
    px_function = 0
    for i in range(0,3):
        px_function = px_function + (coefficient[i] * sp.exp(-alpha[i]*r**2) * r * sp.sin(theta) * sp.cos(phi) *((128 * (alpha[i] ** 5)/(sp.pi ** 3))**(1/4)))
    return px_function

def py_gaussian_function(coefficient,alpha):

    py_function = 0
    for i in range(0,3):
        py_function = py_function + (coefficient[i] * sp.exp(-alpha[i]*r**2) * r * sp.sin(theta) * sp.sin(phi) *((128 * (alpha[i] ** 5)/(sp.pi ** 3))**(1/4)))
    return py_function

def pz_gaussian_function(coefficient,alpha):
   
    pz_function = 0
    for i in range(0,3):
        pz_function = pz_function + (coefficient[i] * sp.exp(-alpha[i]*r**2) * r * sp.cos(theta) *((128 * (alpha[i] ** 5)/(sp.pi ** 3))**(1/4)))
    return pz_function


定义H原子和C原子的STO-3G基组对应的基函数

In [9]:
H_1s = s_gaussian_function(alphas_1s_H, coefficients_1s_H)

C_1s = s_gaussian_function(alphas_1s_C, coefficients_1s_C)
C_2s = s_gaussian_function(alphas_2s_C, coefficients_2s_C)
C_2px = px_gaussian_function(alphas_2p_C, coefficients_2p_C)
C_2py = py_gaussian_function(alphas_2p_C, coefficients_2p_C)
C_2pz = pz_gaussian_function(alphas_2p_C, coefficients_2p_C)

H_basis = H_1s
C_basis = [C_1s, C_2s, C_2px, C_2py, C_2pz]

In [93]:
print(H_1s)

0.656692559629637*exp(-0.5353281423*r**2)/pi**0.75 + 0.154628612852294*exp(-0.4446345422*r**2)/pi**0.75 + 1.41840572401755*exp(-0.1543289673*r**2)/pi**0.75


In [94]:
print(C_2px)

1.23346625175088*r*exp(-0.6076837186*r**2)*sin(theta)*cos(phi)/pi**0.75 + 0.231884024698061*r*exp(-0.3919573931*r**2)*sin(theta)*cos(phi)/pi**0.75 + 0.969278658257632*r*exp(-0.155916275*r**2)*sin(theta)*cos(phi)/pi**0.75


计算重叠积分矩阵元
$$ S_{\mu\nu}=\int\phi_\mu^*(\bm{r})\phi_\nu(\bm{r})\mathrm{d}\bm{r}$$


In [106]:
def overlap_integral(basis1,basis2):

    intf = basis1 * basis2
    intf_numeric=sp.lambdify((r,theta,phi),intf,'numpy')
    
    result, error= scipy.integrate.tplquad(intf_numeric,
                                            0, np.inf,
                                            lambda r:0 , lambda r:np.pi,
                                            lambda r,theta:0 , lambda r,theta:2*np.pi)
    return result

将重叠积分矩阵幺正化，计算变换矩阵X

In [1]:
import scipy.linalg


def overlap_matrix_inverse_sqrt(atom_basis):
    
    overlap_matrix = np.zeros((len(atom_basis),len(atom_basis)))
    for i in range(len(atom_basis)):
        for j in range(len(atom_basis)):
            overlap_matrix[i][j]=overlap_integral(atom_basis[i],atom_basis[j])

    overlap_matrix_inverse_sqrt_ij = scipy.linalg.sqrtm(scipy.linalg.inv(overlap_matrix))
    return overlap_matrix_inverse_sqrt_ij

计算动能积分
$$T_{\mu\nu}=\int \phi_\mu^*(\bm{r})(-\frac{1}{2}\nabla^2)\phi_\nu(\bm{r})\mathrm{d}\bm{r}$$
在球坐标下
$$\nabla^2=-\frac{1}{r^2}\frac{\partial}{\partial r}(r^2\frac{\partial }{\partial r})+\frac{1}{r^2 \sin\theta}\frac{\partial}{\partial \theta}(\sin\theta\frac{\partial}{\partial\theta})+\frac{1}{r^2 \sin^2\theta}\frac{\partial^2}{\partial\phi^2}$$

In [97]:
def kinetic_energy(basis1,basis2):
    
    
    intf = -0.5*basis1*((-1/r**2)*sp.diff(r**2*sp.diff(basis2,r),r)+1/(r**2*sp.sin(theta))*sp.diff(sp.sin(theta)*sp.diff(basis2,theta),theta)+1/(r**2*sp.sin(theta)**2)*sp.diff(basis2,phi,phi))
    intf_numeric=sp.lambdify((phi,theta,r),intf,'numpy')
    
    result, error = scipy.integrate.tplquad(intf_numeric,
                                            0, 2*np.pi,
                                            lambda phi:0 , lambda phi:np.pi,
                                            lambda r,phi:0 , lambda r,phi:np.inf)
    return result

In [98]:
print(kinetic_energy(H_1s,H_1s))
print(kinetic_energy(C_2px, C_2px))

c:\Users\28724\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\integrate\_quadpack_py.py:1260: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


2.2271097965217685
540.0998133147342


计算核吸引势能
$$V_{\mu\nu}=\int \phi_\mu^*(\bm{r})(-\frac{Z_{atom}}{r})\phi_\nu(\bm{r})\mathrm{d}\bm{r}$$

In [17]:
def nuclear_attraction_energy(basis1,basis2,nuclear_number):
    
    intf = -basis1*basis2*nuclear_number/r
    intf_numeric=sp.lambdify((r,theta,phi),intf,'numpy')
    
    result, error = scipy.integrate.tplquad(intf_numeric,
                                            0, np.inf,
                                            lambda r:0 , lambda r:np.pi,
                                            lambda r,theta:0 , lambda r,theta:2*np.pi)
    return result


计算双电子积分中的库仑积分

$$K_{ij}=\int \phi_\mu^*(\bm{r}_1)\phi_\nu(\bm{r}_1)\frac{1}{|\bm{r}_1-\bm{r}_2|}\phi_i^*(\bm{r}_2)\phi_j(\bm{r}_2) \mathrm{d}\bm{r}_1\mathrm{d}\bm{r}_2$$

In [16]:
def coulomb_energy(basis1,basis2,basis3,basis4):

    basis21=basis2.subs({r:r1, theta:theta1, phi:phi1})
    basis41=basis4.subs({r:r1, theta:theta1, phi:phi1})

    intf=basis1*basis21*basis3*basis41/sp.sqrt(r**2+r1**2-2*r*r1(sp.sin(phi)*sp.sin(phi1)*sp.sin(theta-theta1)+sp.cos(phi)*sp.cos(phi1)))

    intf_numeric=sp.lambdify((r,theta,phi,r1,theta1,phi1),intf,'numpy')

    r_bounds=[0,np.inf]
    theta_bounds=[0,np.pi]
    phi_bounds=[0,2*np.pi]
    r1_bounds=[0,np.inf]
    theta1_bounds=[0,np.pi]
    phi1_bounds=[0,2*np.pi]

    result, error = scipy.integrate.nquad(intf_numeric,
                                          [r_bounds, theta_bounds, phi_bounds, r1_bounds, theta1_bounds, phi1_bounds])
    
    return result


    

下面给出库仑积分的矩阵元

$$K_{\mu\nu}=\sum_i\sum_j K_{ij}$$

In [1]:
def coulomb_energy_matrix(basis1,basis2,atom_basis):
    
    coulomb_energy_uv = 0
    for i in range(len(atom_basis)):
        for j in range(len(atom_basis)):
            coulomb_energy_uv = coulomb_energy_uv + coulomb_energy(basis1,basis2,atom_basis[i],atom_basis[j])

    return coulomb_energy_uv

计算双电子积分中的交换积分

$$J_{ij}=\int \phi_\mu^*(\bm{r}_1)\phi_\nu(\bm{r}_2)\frac{1}{|\bm{r}_1-\bm{r}_2|}\phi_i^*(\bm{r}_2)\phi_j(\bm{r}_1) \mathrm{d}\bm{r}_1\mathrm{d}\bm{r}_2$$

In [101]:
def exchange_energy(basis1,basis2,basis3,basis4):

    basis21=basis2.subs({r:r1, theta:theta1, phi:phi1})
    basis31=basis3.subs({r:r1, theta:theta1, phi:phi1})

    intf=basis1*basis21*basis31*basis4/sp.sqrt(r**2+r1**2-2*r*r1(sp.sin(phi)*sp.sin(phi1)*sp.sin(theta-theta1)+sp.cos(phi)*sp.cos(phi1)))

    intf_numeric=sp.lambdify((r,theta,phi,r1,theta1,phi1),intf,'numpy')

    r_bounds=[0,np.inf]
    theta_bounds=[0,np.pi]
    phi_bounds=[0,2*np.pi]
    r1_bounds=[0,np.inf]
    theta1_bounds=[0,np.pi]
    phi1_bounds=[0,2*np.pi]

    result, error = scipy.integrate.nquad(intf_numeric,
                                          [r_bounds, theta_bounds, phi_bounds, r1_bounds, theta1_bounds, phi1_bounds])
    
    return result

给出交换积分的矩阵元

$$J_{\mu\nu}=\sum_i\sum_j J_{ij}$$

In [15]:
def exchange_energy_matrix(basis1,basis2,atom_basis):
    
    exchange_energy_uv = 0
    
    for i in range(len(atom_basis)):
        for j in range(len(atom_basis)):
            exchange_energy_uv = exchange_energy_uv + exchange_energy(basis1,basis2,atom_basis[i],atom_basis[j])

    return exchange_energy_uv

根据上面计算的积分给出UHF中的Fock矩阵
$$F^\alpha_{\mu\nu}=T_{\mu\nu}+V_{\mu\nu}+P_{\mu\nu}J_{\mu\nu}-P^\alpha_{\mu\nu}K_{\mu\nu} $$
$$F^\beta_{\mu\nu}=T_{\mu\nu}+V_{\mu\nu}+P_{\mu\nu}J_{\mu\nu}-P^\beta_{\mu\nu}K_{\mu\nu} $$


In [4]:
def fock_matrix(atom_basis,density_matrix_alpha,density_matrix_beta,electron_number_alpha,electron_number_beta):

    fock_matrix_alpha = np.zeros((len(atom_basis),len(atom_basis)))
    fock_matrix_beta = np.zeros((len(atom_basis),len(atom_basis)))
    for i in range(len(atom_basis)):
        for j in range(len(atom_basis)):
            density = 0, density_alpha = 0, density_beta = 0
            for k in range(len(electron_number_alpha)):
                density_alpha = density_alpha + density_matrix_alpha[i][k]*density_matrix_alpha[j][k]
            for k in range(len(electron_number_beta)):
                density_beta = density_beta + density_matrix_beta[i][k]*density_matrix_beta[j][k]
            density = density_alpha + density_beta
            fock_matrix_alpha[i][j] = kinetic_energy(atom_basis[i],atom_basis[j]) + nuclear_attraction_energy(atom_basis[i],atom_basis[j])+density*coulomb_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)-density_alpha*exchange_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)
            fock_matrix_beta[i][j] = kinetic_energy(atom_basis[i],atom_basis[j]) + nuclear_attraction_energy(atom_basis[i],atom_basis[j])+density*coulomb_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)-density_beta*exchange_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)
    return fock_matrix_alpha, fock_matrix_beta

SCF自洽循环过程

1.产生密度矩阵P的初始猜测，这里生成两个单位矩阵，分别对于$\alpha$电子和$\beta$电子

2.由密度矩阵P计算Fock矩阵F

3.计算幺正化基组对应的Fock矩阵F'=X $^\dagger$FX

4.对角化F'，得到特征值（轨道能）特征向量C'

5.计算轨道展开系数C=XC'

6.计算新的密度矩阵P

7。判断是否收敛，未收敛则返回第二步

In [2]:
def SCF_cycle(atom_basis,electron_number_alpha,electron_number_beta):
    
    #初始化两个单位阵
    density_matrix_alpha = np.eye(len(atom_basis))
    density_matrix_beta = np.eye(len(atom_basis))
    iteration_max = 1e4
    for i in range(iteration_max):
        fock_matrix_alpha,fock_matrix_beta=fock_matrix(atom_basis,electron_number_alpha,electron_number_beta)
        fock_matrix_alpha1 = np.dot(overlap_matrix_inverse_sqrt(atom_basis),np.dot(fock_matrix_alpha,overlap_matrix_inverse_sqrt(atom_basis)))
        fock_matrix_beta1 = np.dot(overlap_matrix_inverse_sqrt(atom_basis),np.dot(fock_matrix_beta,overlap_matrix_inverse_sqrt(atom_basis)))
        eigenvalues1, eigenvectors1 = np.linalg.eigh(fock_matrix_alpha1)
        eigenvalues2, eigenvectors2 = np.linalg.eigh(fock_matrix_beta1)
        density_matrix_alpha_new = np.dot(overlap_matrix_inverse_sqrt(atom_basis),eigenvectors1)
        density_matrix_beta_new = np.dot(overlap_matrix_inverse_sqrt(atom_basis),eigenvectors2)
        if abs(np.linalg.norm(density_matrix_alpha_new)+np.linalg.norm(density_matrix_beta_new)-(np.linalg.norm(density_matrix_alpha)+np.linalg.norm(density_matrix_beta)))<1e-4:
            break
        else:
            density_matrix_alpha = density_matrix_alpha_new
            density_matrix_beta = density_matrix_beta_new


    
    


In [ ]:
SCF_cycle(H_basis,1,0)
SCF_cycle(C_basis,4,2)

遗憾的是，这个直接使用简单粗暴的数值积分的代码计算量太大了，实际上很难运行起来。使用Gauss-Legendre网格来计算积分可能会效率更高，但是我目前还写不出来，尽力了：(
如果要使用DFT（LDA）方法计算，只需要把交换能更换为以下形式：
$$E_x^{LDA}[\rho]=-\frac{3}{4}(\frac{3}{\pi})^{\frac{1}{3}}\int \rho(r)^{4/3} dr $$

In [ ]:
def exchange_correlation_energy_LDA(rho):
    
    C_lda = (3/4) * (3.0/np.pi) ** (1/3)
    return -C_lda * (rho) ** (4/3)

In [5]:
def fock_matrix(atom_basis,density_matrix_alpha,density_matrix_beta,electron_number_alpha,electron_number_beta):

    fock_matrix_alpha = np.zeros((len(atom_basis),len(atom_basis)))
    fock_matrix_beta = np.zeros((len(atom_basis),len(atom_basis)))
    for i in range(len(atom_basis)):
        for j in range(len(atom_basis)):
            density = 0
            density_alpha = 0
            density_beta = 0
            for k in range(len(electron_number_alpha)):
                density_alpha = density_alpha + density_matrix_alpha[i][k]*density_matrix_alpha[j][k]
            for k in range(len(electron_number_beta)):
                density_beta = density_beta + density_matrix_beta[i][k]*density_matrix_beta[j][k]
            density = density_alpha + density_beta
            fock_matrix_alpha[i][j] = kinetic_energy(atom_basis[i],atom_basis[j]) + nuclear_attraction_energy(atom_basis[i],atom_basis[j])+density*coulomb_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)-exchange_correlation_energy_LDA(density)
            fock_matrix_beta[i][j] = kinetic_energy(atom_basis[i],atom_basis[j]) + nuclear_attraction_energy(atom_basis[i],atom_basis[j])+density*coulomb_energy_matrix(atom_basis[i],atom_basis[j],atom_basis)-exchange_correlation_energy_LDA(density)
    return fock_matrix_alpha, fock_matrix_beta